# Submitting data

- Making an interactive service means that your users can receive and send information. We use GET requests to get a page and POST, PUT, PATCH, DELETE requests to update data on the server side. It's not a requirement, but a rule to follow. You can still make a valid application if you break it, but its behavior would be unexpected for its users and developers alike.

# Query parameters

For example, searching by keyword may look like www.makeawish.happen/Mymble?q=cake. We use a query string after the question mark to specify the parameters and their values. To access these values in a request handler, we can use the GET attribute of the HttpRequest class.

- The q parameter in the GET attribute can now be accessed as a key in a dictionary because this attribute is a QueryDict instance. QueryDict is a subclass of Python's dictionary, but the main difference is that QueryDict is immutable; you can't modify the user's request, which makes sense.

# Request body

All requests have a body. Sometimes it's empty, but for POST requests, it's filled with data. In these cases, you can find what the client sent to the server in the POST attribute of the HttpRequest class.

How does a client create the request body? You cannot find it in the URL, but you can pass it in an HTML form. Assume that we have the following form:


In [ ]:
<form action="/Mymble" method="post">
    {% csrf_token %}
    <input name="wish">
    <button type="submit">Add a wish</button>
  </form>

The wish parameter is stored in the POST attribute. This attribute is also a QueryDict instance, so use it as a usual dictionary; still, do not change anything in it, you will get an AttributeError exception. Let us move on to making a handler and processing a new wish.


'We have only GET and POST attributes in the HttpRequest class.'

To make this example work, create the WishForm class in the ```forms.py``` file and an HTML template with the name ```form_template.html.```

# POST request

Making a wish is the first step in bringing it to life. We use the POST handler to save a wish in the application; the signature of a method is similar to the GET handler. The lists are stored only in a static attribute of a class — let's see an example of ```views.py:```

In [ ]:
from collections import defaultdict

from django.shortcuts import redirect
from django.views import View

class WishListView(View):
   form_class = WishForm
   initial = {'key': 'value'}
   template_name = 'form_template.html'
 
   def get(self, request, *args, **kwargs):
       form = self.form_class(initial=self.initial)
       return render(request, self.template_name, {'form': form})
 
   wishlist = defaultdict(list)
 
   def post(self, request, *args, **kwargs):
      author = 'Anonymous'
      if request.user.is_authenticated:
           author = request.user.username
      form = self.form_class(request.POST)
      if form.is_valid():
           wish = form.cleaned_data['wish']
           self.wishlist[author].append(wish)
           return redirect('/')

request.user.is_authenticated is the request attribute that represents the current user and whether they are authenticated. The wishes are grouped by the author; if a user is not authorized, their wish goes to the Anonymous group. When a wish is saved, the handler redirects the client to the / page

# DELETE request

Fleeting wishes come and go; when they go, our user would probably want to remove them from the wishlist. The most appropriate method for this task is DELETE.

In [ ]:
from collections import defaultdict

from django.shortcuts import redirect, Http404
from django.views import View


class WishListView(View):
    wishlist = defaultdict(list)
 
    def delete(self, request, wish, *args, **kwargs):
        author = 'Anonymous'
        if request.user.is_authenticated:
            author = request.user.username
 
        if wish in self.wishlist[author]:
            self.wishlist[author].remove(wish)
        else:
             raise Http404
        return redirect('/')
 
    def get(self, request, *args, **kwargs):
        …
    def post(self, request, *args, **kwargs):
        ...